In [1]:
!pip install streamlit 

In [2]:
!pip install pyngrok

In [3]:
%%writefile heath.py

import streamlit as st

# Configure the app layout
st.set_page_config(page_title="Health Tracker App", layout="centered")

# Custom CSS for styling
st.markdown(
    """
    <style>
    .stButton > button {
        width: 200px;
        height: 50px;
        font-size: 16px;
        font-weight: bold;
        border-radius: 10px;
        margin-top: 10px;
    }
    .category-container {
        text-align: center;
        margin: 20px;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

# Main Page
st.title("Health Tracker App")
st.subheader("Built by 10xTechClub C2 Students")

# Categories with Images and Buttons
st.markdown("### Choose a Category")

# Align Categories in a Horizontal Row
col1, col2, col3 = st.columns([1, 1, 1])  # Equal-sized columns for perfect alignment

# Nutrition with col1:
with col1:
    st.image("nutrition.png", caption="Nutrition", width=120)
    if st.button("Nutrition"):
        st.session_state['page'] = 'nutrition'

# Fitness with col2:
with col2:
    st.image("fitness.png", caption="Fitness", width=120)
    if st.button("Fitness"):
        st.session_state['page'] = 'fitness'

# Sleep with col3:
with col3:
    st.image("sleeping.png", caption="Sleep", width=120)
    if st.button("Sleep"):
        st.session_state['page'] = 'sleep'

# Handle Page Navigation
if 'page' in st.session_state:
    if st.session_state['page'] == 'nutrition':
        st.write("**Opening Nutrition App...**")
        # Code to launch Nutrition App
    elif st.session_state['page'] == 'fitness':
        st.write("**Opening Fitness Subcategories...**")
        st.markdown("### Choose a Fitness Option")

        # Buttons for different Fitness Options in a single row
        col1, col2, col3 = st.columns([1, 1, 1])  # Equal-sized columns for perfect alignment

        with col1:
            if st.button("Strava App"):
                st.write("**Opening Strava App...**")
                # Code to launch Strava App

        with col2:
            if st.button("Step Counter"):
                st.write("**Opening Step Counter App...**")
                # Code to launch Step Counter App

        with col3:
            if st.button("Workout Planner"):
                st.write("**Opening Workout Planner App...**")
                # Code to launch Workout Planner

    elif st.session_state['page'] == 'sleep':
        st.write("**Opening Sleep Tracker App...**")
        # Code to launch Sleep Tracker


Overwriting heath.py


## Google fit  code  

In [20]:
%%writefile googlefit.py

import streamlit as st
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
import google_auth_oauthlib.flow
import googleapiclient.discovery

# Path to your client secret JSON file
CLIENT_SECRETS_FILE = "client_secret.json"

# The scopes required to access Fit data
SCOPES = [
    "https://www.googleapis.com/auth/fitness.activity.read",
    "https://www.googleapis.com/auth/fitness.location.read",
    "https://www.googleapis.com/auth/fitness.body.read"
]

# API version
API_SERVICE_NAME = 'fitness'
API_VERSION = 'v1'

# Authentication flow
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    CLIENT_SECRETS_FILE, SCOPES)
credentials = flow.run_local_server(port=0)

# Build the API service
service = googleapiclient.discovery.build(
    API_SERVICE_NAME, API_VERSION, credentials=credentials)

# Fetch steps data for the last 7 days
now = datetime.now(timezone.utc)
start_time = int((now - timedelta(days=7)).timestamp() * 1000)
end_time = int(now.timestamp() * 1000)

dataset = f"{start_time}-{end_time}"

# Request step data
response = service.users().dataset().aggregate(userId="me", body={
    "aggregateBy": [{
        "dataTypeName": "com.google.step_count.delta"
    }],
    "bucketByTime": {"durationMillis": 86400000},  # Daily steps
    "startTimeMillis": start_time,
    "endTimeMillis": end_time
}).execute()

# Initialize lists for plotting
dates = []
steps_counts = []
total_calories_per_date = {}
calories_per_step = 0.05  # Average calories burned per step

# Extract data for plotting
for bucket in response.get('bucket', []):
    steps = 0
    start_time = int(bucket['startTimeMillis'])
    date = datetime.utcfromtimestamp(start_time / 1000).strftime('%Y-%m-%d')

    for dataset in bucket.get('dataset', []):
        for point in dataset.get('point', []):
            for value in point.get('value', []):
                steps += value.get('intVal', 0)

    # Calculate estimated calories burned
    estimated_calories = steps * calories_per_step

    # Store data for plotting
    dates.append(date)
    steps_counts.append(steps)

    # Accumulate calories for the same date
    total_calories_per_date[date] = total_calories_per_date.get(date, 0) + estimated_calories

# Prepare data for plotting calories
calories_burned = list(total_calories_per_date.values())

# Streamlit app layout
st.title("Fitness Data Overview")

# Step Count Plot
st.header("Step Count Over Time")
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(dates, steps_counts, marker='o', linestyle='-', color='b')
ax.set_title('Step Count Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Step Count')
ax.grid(True)
st.pyplot(fig)

# Calories Burned Plot
st.header("Estimated Calories Burned by Date")
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(dates, calories_burned, color='green')
ax.set_title('Estimated Calories Burned by Date')
ax.set_xlabel('Date')
ax.set_ylabel('Estimated Calories Burned')
ax.grid(axis='y')
st.pyplot(fig)


Overwriting googlefit.py


## Sleep tracker

In [10]:
%%writefile sleep.py
import streamlit as st
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from pathlib import Path

# File to store sleep data
DATA_FILE = "sleep_data.json"
data_path = Path(DATA_FILE)

# Load or initialize sleep data
if data_path.exists():
    with open(DATA_FILE, "r") as f:
        sleep_data = json.load(f)
else:
    sleep_data = {}

    
# Function to save data to JSON
def save_data():
    with open(DATA_FILE, "w") as f:
        json.dump(sleep_data, f)

# Sidebar for navigation
st.sidebar.header("Sleep Tracker Menu")
option = st.sidebar.radio("Choose an option:", ["Log Sleep", "Streak Counter", "Bar Graph"])

# Desired sleep schedule
st.sidebar.header("Set Desired Sleep Schedule")
start_time = st.sidebar.time_input("Desired Sleep Time", value=datetime.strptime("10:00 PM", "%I:%M %p").time())
wake_time = st.sidebar.time_input("Desired Wake-Up Time", value=datetime.strptime("06:00 AM", "%I:%M %p").time())

# Calculate target sleep duration in hours
target_sleep_duration = (datetime.combine(datetime.today(), wake_time) - datetime.combine(datetime.today(), start_time)).seconds / 3600

# Initialize session state for sleep data logging
if "sleep_logged" not in st.session_state:
    st.session_state["sleep_logged"] = False

# Log sleep data function
def log_sleep_data():
    selected_date = st.date_input("Select Sleep Start Date", datetime.now())
    sleep_start = st.time_input("Sleep Start Time (AM/PM)", key="sleep_start")
    wake_up_date = st.date_input("Select Wake-Up Date", selected_date + timedelta(days=1))
    wake_up_time = st.time_input("Wake-Up Time (AM/PM)", key="wake_up_time")

    if st.button("Save Sleep Routine"):
        try:
            # Combine selected date with times for accurate duration calculation
            sleep_start_dt = datetime.combine(selected_date, sleep_start)
            wake_up_dt = datetime.combine(wake_up_date, wake_up_time)
            
            # Ensure wake-up time is later than sleep start time
            if wake_up_dt <= sleep_start_dt:
                st.error("Wake-up time must be after sleep start time. Please adjust the dates or times.")
                return

            # Calculate the duration in hours
            hours_slept = (wake_up_dt - sleep_start_dt).total_seconds() / 3600
            
            # Use wake-up date as the key for JSON to reflect the day the user woke up
            date_key = wake_up_date.isoformat()
            sleep_data[date_key] = {
                "sleep_start": sleep_start_dt.isoformat(),
                "wake_up": wake_up_dt.isoformat(),
                "hours_slept": hours_slept
            }
            save_data()
            
            st.session_state["sleep_logged"] = True
            st.success(f"Logged {hours_slept:.2f} hours for wake-up on {wake_up_date}")
        except Exception as e:
            st.error(f"Error logging sleep data: {e}")

# Weekly streak display
def display_weekly_schedule():
    current_date = datetime.now().date()
    streak_count = 0
    weekly_data = []

    for i in range(7):
        day = current_date - timedelta(days=i)
        date_str = day.isoformat()
        hours_slept = sleep_data.get(date_str, {}).get("hours_slept", 0)

        color = "green" if hours_slept >= target_sleep_duration else "red"
        if color == "green":
            streak_count += 1
        weekly_data.append((day.strftime("%A %Y-%m-%d"), hours_slept, color))

    # Display streak
    st.write(f"Streak: {streak_count} days this week")
    for date, hours, color in weekly_data[::-1]:
        st.markdown(f"<span style='color:{color}'>{date}: {hours} hours</span>", unsafe_allow_html=True)

# Plot sleep graph
def plot_sleep_graph():
    dates = sorted(sleep_data.keys())
    hours = [sleep_data[date]["hours_slept"] for date in dates]

    if dates and hours:
        plt.figure(figsize=(10, 5))
        plt.bar(dates, hours, color="skyblue")
        plt.axhline(y=target_sleep_duration, color="green", linestyle="--", label="Target Sleep")
        plt.xlabel("Date")
        plt.ylabel("Hours Slept")
        plt.title("Sleep Tracker")
        plt.xticks(rotation=45)
        plt.legend()
        st.pyplot(plt)
        
    else:
        st.info("No sleep data available to plot.")

# Main App Logic
st.title("Sleep Tracker")

# Render sections based on selected option
if option == "Log Sleep":
    st.header("Log Today's Sleep")
    log_sleep_data()
elif option == "Streak Counter":
    st.header("Weekly Streak Counter")
    display_weekly_schedule()
elif option == "Bar Graph":
    st.header("Sleep Hours Graph")
    plot_sleep_graph()


Overwriting sleep.py


In [18]:
%%writefile workoutplanner.py

import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import streamlit as st

# Initialize Streamlit app
st.title("Personalized Workout Planner")

# ExerciseDB API credentials
api_url = "https://exercisedb.p.rapidapi.com/exercises"
headers = {
    "x-rapidapi-key": "fcd1ee5e87msha255e9b87002774p1dfe1bjsn0106ef4dd44b",
    "x-rapidapi-host": "exercisedb.p.rapidapi.com"
}

# Fetch exercises from API


def fetch_exercises():
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        st.error(f"Failed to fetch data: {response.status_code}")
        return []

# Define calorie burn rates per exercise type (hypothetical values)
calorie_burn_rates = {
    "cardio": 10,  # calories per minute
    "strength": 8,
    "flexibility": 5
}

# Filter exercises based on goal
def filter_exercises_by_goal(exercises, goal):
    if goal == "weight_loss":
        return [e for e in exercises if e['bodyPart'] == 'cardio']
    elif goal == "muscle_gain":
        return [e for e in exercises if e['bodyPart'] in ['upper arms', 'chest', 'back', 'lower legs']]
    elif goal == "fitness":
        return exercises
    else:
        return []

# Generate workout plan
def generate_workout_plan(exercises, goal, duration):
    workout_plan = []
    for exercise in exercises[:5]:  # Select first 5 exercises as a sample workout
        exercise_type = "cardio" if exercise['bodyPart'] == 'cardio' else "strength"
        calories_burned = calorie_burn_rates.get(exercise_type, 5) * duration
        muscle_gain_points = 3 if exercise_type == "strength" else 1
        workout_plan.append({
            "name": exercise.get('name', 'Unnamed Exercise'),
            "type": exercise_type,
            "target": exercise.get('target', 'Unknown'),
            "calories_burned": calories_burned,
            "duration": duration,
            "muscle_gain_points": muscle_gain_points
        })
    return workout_plan

# Visualize workout plan with charts
def visualize_workout_plan(workout_plan, goal):
    # Ensure workout_plan has the expected structure
    if not workout_plan:
        st.error("Workout plan is empty. Please try again with different parameters.")
        return

    # Convert workout plan to DataFrame and display for debugging
    try:
        df = pd.DataFrame(workout_plan)
        st.write("### Debugging DataFrame Content")
        st.write(df)
    except Exception as e:
        st.error(f"Failed to create DataFrame: {e}")
        return

    # Ensure 'calories_burned' column exists
    if 'calories_burned' not in df.columns:
        st.error("'calories_burned' column is missing from the DataFrame.")
        return

    # Total calories burned
    total_calories = df['calories_burned'].sum()
    cumulative_calories = df['calories_burned'].cumsum()

    # Bar chart for calories burned per exercise
    st.subheader(f"Calories Burned per Exercise for {goal.capitalize()}")
    plt.figure(figsize=(10, 6))
    sns.barplot(x=df['name'], y=df['calories_burned'], palette="coolwarm")
    plt.xticks(rotation=45)
    plt.xlabel("Exercises")
    plt.ylabel("Calories Burned")
    st.pyplot(plt)

    # Line chart for cumulative calories burned
    st.subheader("Cumulative Calories Burned - Fat Reduction Progress")
    plt.figure(figsize=(10, 6))
    plt.plot(df['name'], cumulative_calories, marker='o', linestyle='-', color='coral')
    plt.xlabel("Exercises")
    plt.ylabel("Cumulative Calories Burned")
    plt.xticks(rotation=45)
    st.pyplot(plt)

    # Pie chart for exercise target distribution
    st.subheader("Exercise Target Distribution")
    target_counts = df['target'].value_counts()
    plt.figure(figsize=(8, 8))
    plt.pie(target_counts, labels=target_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
    st.pyplot(plt)

    # Line chart for muscle gain potential
    st.subheader("Muscle Gain Potential Over Workout")
    plt.figure(figsize=(10, 6))
    plt.plot(df['name'], df['muscle_gain_points'].cumsum(), marker='o', linestyle='-', color='slateblue')
    plt.xlabel("Exercises")
    plt.ylabel("Cumulative Muscle Gain Points")
    plt.xticks(rotation=45)
    st.pyplot(plt)

    st.write(f"**Total Calories Burned:** {total_calories} kcal")

# Main program for Streamlit
exercises = fetch_exercises()

# Display available goals with dropdown
goal = st.selectbox("Select Your Goal", ("weight_loss", "muscle_gain", "fitness"))

# Duration input for calorie calculation
duration = st.number_input("Enter workout duration per exercise (in minutes)", min_value=1, max_value=120, value=30)

# Filter exercises by goal
filtered_exercises = filter_exercises_by_goal(exercises, goal)

# Generate workout plan button
if st.button("Generate Workout Plan"):
    workout_plan = generate_workout_plan(filtered_exercises, goal, duration)

    # Display workout plan
    st.write("### Your Workout Plan:")
    for exercise in workout_plan:
        st.write(f"- {exercise['name']} ({exercise['type']}, Target: {exercise['target']}) - {exercise['duration']} mins, {exercise['calories_burned']} kcal burned")

    # Visualize workout plan with charts
    visualize_workout_plan(workout_plan, goal)


Overwriting workoutplanner.py


## neutrition tracker


In [5]:
%%writefile Nutrition.py

import streamlit as st
import requests
import pandas as pd
import plotly.express as px

# Nutritionix API credentials
# Nutritionix API credentials
API_ID = "00b1404a"  # Replace with your actual APP_ID
API_KEY = "b563211867bc021d51476b07eeab5a27"

# Initialize session state for food log
if "food_log" not in st.session_state:
    st.session_state.food_log = []

# Fetch data from Nutritionix API
def fetch_nutrition_data(query):
    url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
    headers = {
        "x-app-id": API_ID,
        "x-app-key": API_KEY,
        "Content-Type": "application/json",
    }
    data = {"query": query}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        st.error("Error fetching data. Please try again.")
        return None

# Generate suggestions
def generate_suggestions(macros, water_intake):
    suggestions = []
    # Calculate the carb-to-protein ratio
    carb_protein_ratio = macros["Carbohydrates (g)"] / (macros["Protein (g)"] + 1e-9)

    # High carb-to-protein ratio suggestion
    if carb_protein_ratio > 2.5:
        suggestions.append("Consider increasing your protein intake to maintain a better macronutrient balance.")
        suggestions.append("Here are some high-protein meal options to include:")
        suggestions.append("- Breakfast: Methi Jowar Thepla (2 Small) + Green Chutney (2 Tbsp) + Onion Tomato Cucumber Curd Raita (1 Small Bowl)")
        suggestions.append("- Lunch: Multigrain Roti (1 Medium) + Paneer Pulao/Egg Pulao (1 Medium Bowl) + Sev Tamatu Nu Shaak (1 Small Bowl) + Curd (1 Small Bowl)")
        suggestions.append("- Dinner: Multigrain Roti (1 Small) + Paneer Kofta Kadhi (1 Medium Bowl) + Rice (1 Small Bowl) + Sweet Curd (1 Small Bowl)")

    # Low carb-to-protein ratio suggestion
    elif carb_protein_ratio < 1.5:
        suggestions.append("Your carbohydrate intake is relatively low; consider adding more carbs like whole grains or fruits.")

    # High fat suggestion
    if macros["Fat (g)"] > 70:
        suggestions.append("Your fat intake is high. Focus on consuming healthy fats like avocados, nuts, and olive oil.")

    # Water intake suggestions
    if water_intake < 4.0:
        suggestions.append("Increase your water intake to at least 2 liters per day for better hydration.")
    elif water_intake > 3.5:
        suggestions.append("Your water intake is high; avoid excessive hydration to maintain electrolyte balance.")

    return suggestions



# Main Streamlit app
def main():
    st.set_page_config(page_title="Daily Health Tracker App", layout="wide")

    # Sidebar for user input
    st.sidebar.header("Log Your Daily Food Intake")
    food_item = st.sidebar.text_input("Food Item (e.g., apple, bread):", "")

    # Dropdown for unit of measurement (e.g., cup, slice, etc.)
    unit_of_measurement = st.sidebar.selectbox(
        "Unit of Measurement",
        ["cup", "slice", "gram", "piece", "tablespoon", "teaspoon", "other"]
    )

    # Number input for quantity
    quantity = st.sidebar.number_input("Quantity (e.g., 1, 2, 3):", min_value=1, step=1, value=1)

    brand = st.sidebar.text_input("Brand (Optional):", "")
    water_intake = st.sidebar.number_input("Water Intake (in liters):", min_value=0.0, step=0.1, value=0.0)
    food_type = st.sidebar.selectbox("Type of Food:", ["Regular", "Organic", "Vegan", "Gluten-Free", "Other"])
    add_button = st.sidebar.button("Add to Log")

    st.title("Daily Nutrition Tracker App")
    st.markdown("""
    This app allows you to log your daily food intake and track nutrition information.

    At the end of the day, generate a comprehensive report with personalized suggestions.
    """)

    # Add food to log
    if add_button and food_item:
        query = f"{quantity} {unit_of_measurement} {food_item}" + (f" {brand}" if brand else "")
        data = fetch_nutrition_data(query)
        if data:
            food_data = data["foods"][0]
            st.session_state.food_log.append({
                "Food Name": food_data["food_name"],
                "Quantity": food_data["serving_qty"],
                "Unit": food_data["serving_unit"],
                "Calories": food_data["nf_calories"],
                "Carbohydrates (g)": food_data["nf_total_carbohydrate"],
                "Protein (g)": food_data["nf_protein"],
                "Fat (g)": food_data["nf_total_fat"],
                "Cholesterol (mg)": food_data.get("nf_cholesterol", 0),
                "Sodium (mg)": food_data.get("nf_sodium", 0),
                "Potassium (mg)": food_data.get("nf_potassium", 0),
                "Type": food_type,
            })
            st.success(f"{food_item} added to your daily log!")

    # Display the food log
    if st.session_state.food_log:
        st.subheader("Daily Food Log")
        food_log_df = pd.DataFrame(st.session_state.food_log)
        st.dataframe(food_log_df)

        # Generate Final Report Button
        if st.button("Generate Final Report"):
            st.header("Final Daily Report")

            # Aggregate Data
            summary = food_log_df[["Calories", "Carbohydrates (g)", "Protein (g)", "Fat (g)"]].sum()
            water_total = water_intake

            st.subheader("Summary of Intake")
            st.write(summary)

            # Macronutrient Pie Chart
            st.subheader("Macronutrient Distribution")
            pie_fig = px.pie(
                names=["Carbohydrates", "Protein", "Fat"],
                values=summary[["Carbohydrates (g)", "Protein (g)", "Fat (g)"]],
                title="Macronutrient Distribution"
            )
            st.plotly_chart(pie_fig)

            # Micronutrient Bar Chart
            micronutrients = food_log_df[["Cholesterol (mg)", "Sodium (mg)", "Potassium (mg)"]].sum()
            bar_fig = px.bar(
                x=micronutrients.index,
                y=micronutrients.values,
                title="Micronutrient Levels",
                labels={"x": "Micronutrient", "y": "Amount"}
            )
            st.subheader("Micronutrient Levels")
            st.plotly_chart(bar_fig)

            # Suggestions
            st.subheader("Personalized Suggestions")
            suggestions = generate_suggestions(summary, water_total)
            if suggestions:
                for suggestion in suggestions:
                    st.write("- " + suggestion)
            else:
                st.write("Great job! Keep maintaining a balanced diet and hydration.")

    else:
        st.info("Your daily food log is empty. Add items from the sidebar to get started.")

if __name__ == "__main__":
    main()


Overwriting Nutrition.py


In [6]:
!pip install streamlit requests pandas plotly

In [8]:
%%writefile nutintegration.py

import streamlit as st
import requests
import pandas as pd
import plotly.express as px

# Configure the app layout
st.set_page_config(page_title="Health Tracker App", layout="centered")

# Custom CSS for styling
st.markdown(
    """
    <style>
    .stButton > button {
        width: 200px;
        height: 50px;
        font-size: 16px;
        font-weight: bold;
        border-radius: 10px;
        margin-top: 10px;
    }
    .category-container {
        text-align: center;
        margin: 20px;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

# Nutritionix API credentials
API_ID = "00b1404a"  # Replace with your actual APP_ID
API_KEY = "b563211867bc021d51476b07eeab5a27"

# Initialize session state for food log
if "food_log" not in st.session_state:
    st.session_state.food_log = []

# Fetch data from Nutritionix API
def fetch_nutrition_data(query):
    url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
    headers = {
        "x-app-id": API_ID,
        "x-app-key": API_KEY,
        "Content-Type": "application/json",
    }
    data = {"query": query}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        st.error("Error fetching data. Please try again.")
        return None

# Generate suggestions
def generate_suggestions(macros, water_intake):
    suggestions = []
    carb_protein_ratio = macros["Carbohydrates (g)"] / (macros["Protein (g)"] + 1e-9)
    if carb_protein_ratio > 2.5:
        suggestions.append("Consider increasing your protein intake for better balance.")
    elif carb_protein_ratio < 1.5:
        suggestions.append("Consider adding more carbohydrates like fruits or grains.")
    if macros["Fat (g)"] > 70:
        suggestions.append("Focus on healthy fats like avocados or nuts.")
    if water_intake < 4.0:
        suggestions.append("Increase your water intake to at least 2 liters daily.")
    return suggestions

# Main Page
st.title("Health Tracker App")
st.subheader("Built by 10xTechClub C2 Students")
st.markdown("### Choose a Category")

# Align Categories in a Horizontal Row
col1, col2, col3 = st.columns([1, 1, 1])

# Nutrition
with col1:
    st.image("nutrition.png", caption="Nutrition", width=120)
    if st.button("Nutrition"):
        st.session_state['page'] = 'nutrition'

# Fitness
with col2:
    st.image("fitness.png", caption="Fitness", width=120)
    if st.button("Fitness"):
        st.session_state['page'] = 'fitness'

# Sleep
with col3:
    st.image("sleeping.png", caption="Sleep", width=120)
    if st.button("Sleep"):
        st.session_state['page'] = 'sleep'

# Handle Page Navigation
if 'page' in st.session_state:
    if st.session_state['page'] == 'nutrition':
        st.header("Daily Nutrition Tracker App")
        food_item = st.text_input("Food Item (e.g., apple, bread):", "")
        unit = st.selectbox("Unit", ["cup", "slice", "gram", "piece", "tablespoon", "teaspoon", "other"])
        quantity = st.number_input("Quantity:", min_value=1, step=1, value=1)
        water_intake = st.number_input("Water Intake (liters):", min_value=0.0, step=0.1, value=0.0)
        if st.button("Add to Log") and food_item:
            query = f"{quantity} {unit} {food_item}"
            data = fetch_nutrition_data(query)
            if data:
                food_data = data["foods"][0]
                st.session_state.food_log.append({
                    "Food Name": food_data["food_name"],
                    "Quantity": food_data["serving_qty"],
                    "Unit": food_data["serving_unit"],
                    "Calories": food_data["nf_calories"],
                    "Carbohydrates (g)": food_data["nf_total_carbohydrate"],
                    "Protein (g)": food_data["nf_protein"],
                    "Fat (g)": food_data["nf_total_fat"],
                })
                st.success(f"{food_item} added to your log!")

        # Display food log
        if st.session_state.food_log:
            st.subheader("Daily Food Log")
            food_log_df = pd.DataFrame(st.session_state.food_log)
            st.dataframe(food_log_df)

            # Generate Final Report
            if st.button("Generate Final Report"):
                summary = food_log_df[["Calories", "Carbohydrates (g)", "Protein (g)", "Fat (g)"]].sum()
                suggestions = generate_suggestions(summary, water_intake)
                st.write("### Macronutrient Summary")
                st.write(summary)
                st.write("### Suggestions")
                for suggestion in suggestions:
                    st.write("- " + suggestion)



Writing nutintegration.py


In [13]:
%%writefile heath.py

import streamlit as st

# Configure the app layout
st.set_page_config(page_title="Health Tracker App", layout="centered")

# Custom CSS for styling
st.markdown(
    """
    <style>
    .stButton > button {
        width: 200px;
        height: 50px;
        font-size: 16px;
        font-weight: bold;
        border-radius: 10px;
        margin-top: 10px;
    }
    .category-container {
        text-align: center;
        margin: 20px;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

# Main Page
st.title("Health Tracker App")
st.subheader("Built by 10xTechClub C2 Students")

# Categories with Images and Buttons
st.markdown("### Choose a Category")

# Align Categories in a Horizontal Row
col1, col2, col3 = st.columns([1, 1, 1])  # Equal-sized columns for perfect alignment

# Nutrition with col1:
with col1:
    st.image("nutrition.png", caption="Nutrition", width=120)
    if st.button("Nutrition"):
        st.session_state['page'] = 'nutrition'

# Fitness with col2:
with col2:
    st.image("fitness.png", caption="Fitness", width=120)
    if st.button("Fitness"):
        st.session_state['page'] = 'fitness'

# Sleep with col3:
with col3:
    st.image("sleeping.png", caption="Sleep", width=120)
    if st.button("Sleep"):
        st.session_state['page'] = 'sleep'

# Handle Page Navigation
if 'page' in st.session_state:
    if st.session_state['page'] == 'nutrition':
        st.write("**Opening Nutrition App...**")
        # Code to launch Nutrition App
    elif st.session_state['page'] == 'fitness':
        st.write("**Opening Fitness Subcategories...**")
        st.markdown("### Choose a Fitness Option")

        # Buttons for different Fitness Options in a single row
        col1, col2, col3 = st.columns([1, 1, 1])  # Equal-sized columns for perfect alignment

        with col1:
            if st.button("Strava App"):
                st.write("**Opening Strava App...**")
                exec(open("strava.py").read())
                # Code to launch Strava App

        with col2:
            if st.button("Step Counter"):
                st.write("**Opening Step Counter App...**")
                exec(open("googlefit.py").read())
                # Code to launch Step Counter App

        with col3:
            if st.button("Workout Planner"):
                st.write("**Opening Workout Planner App...**")
                exec(open("workout_planner.py").read())
                # Code to launch Workout Planner

    elif st.session_state['page'] == 'sleep':
        st.write("**Opening Sleep Tracker App...**")
        exec(open("sleep.py").read())
        # Code to launch Sleep Tracker


Overwriting heath.py
